In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.utilities import ArxivAPIWrapper
import os
from abc import ABC, abstractmethod
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationalRetrievalChain
import ipywidgets as widgets
from IPython.display import display
import fitz
import streamlit as st
#from streamlit_chat import message

In [2]:
OPEN_API_KEY="" #Enter your OpenAI API KEY
class Embedder:
    "Embedding engine to create doc embeddings"
    def __init__(self,engine='OpenAI'):
        """"Specify embedding model
        Args:
        -----------------
        engine:the embedding model."""
        if engine=='OpenAI':
            self.embeddings=OpenAIEmbeddings(api_key=OPEN_API_KEY)
        else:
            raise KeyError("Currently unsupported chat model type")
    def load_process(self,path):
        """Load and process PDF document.
        Args:
        ---------
        path: path of the paper."""
        #load pdf
        loader=PyMuPDFLoader(path)
        documents=loader.load()
        #process pdf
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
        self.documents=text_splitter.split_documents(documents)
    def create_vectorstore(self,store_path):
        """Create vector store for doc Q&A.
        Args:
        -------------------------
        store_path:path of vector store.
        Outputs:
        vectorstore:the created vector store for holding embeddings"""
        if not os.path.exists(store_path):
            print('Embedding not found! Creating new ones')
            self.vectorstore=FAISS.from_documents(self.documents,self.embeddings)
            self.vectorstore.save_local(store_path)
            
        else:
            print("Embeddings found! Loaded the computed ones")
            self.vectorstore=FAISS.load_local(store_path,self.embeddings,allow_dangerous_deserialization=True)
        return self.vectorstore
    def create_summary(self,llm_engine=None):
        """Create paper summary.
        The summary is created by using LangChain's summarize_chain.
        Args:
        --------------
        llm_engine: llm to use.
        Outputs:
        --------------
        summary: the summary of the paper"""

        if llm_engine is None:
            raise KeyError('please specify a LLM engine to perform summarization')
        elif llm_engine=='OpenAI':
            llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0.8,api_key=OPEN_API_KEY)
        else:
            raise KeyError('Currently unsupported chat model type')
            
        #Use LLM to summarize the paper
        chain=load_summarize_chain(llm,chain_type='stuff')
        summary=chain.run(self.documents[:20])
        return summary
    
        

In [3]:
class Chatbot(ABC):
    """Class definition for a single chatbot with memory,created with LangChain. """
    def __init__(self,engine):
        """Initialize the LLM and its associated memory.
        The memory can be an LangChain memory object,or a list of chat history.
        Args:
        --------------------
        engine: llm to be used."""
        #Initialize llm
        if engine=='OpenAI':
            self.llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0.8,api_key=OPEN_API_KEY)
        else:
            raise KeyError("Currently unsupported chat model type")
    @abstractmethod
    def instruct(self):
        """Determine the context of chatbot interaction"""
        pass
    @abstractmethod
    def step(self):
        """Action produced by the chatbot."""
        pass
    @abstractmethod
    def _specify_system_message(self):
        """Prompt engineering for chatbot."""
        pass
    
        

In [4]:
class JournalistBot(Chatbot):
    """Class definition for the journalist bot,created with Langchain."""
    def __init__(self,engine):
        """Setup journalist bot.
        Args:
        -------------------------
        engine: llm model to be used."""
        #Instantiate llm
        super().__init__(engine)
        #Instantiate memory
        self.memory=ConversationBufferMemory(return_messages=True)
    def instruct(self,topic,abstract):
        """Determine the context of journalist chatbot.
        Args:
        -----------
        topic: the topics of the paper
        abstract: the abstract of the paper."""
        self.topic=topic
        self.abstract=abstract
        #Define prompt template
        prompt=ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self._specify_system_message()),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("""{input}""")
            
        ])
        #Create conversation chain
        self.conversation=ConversationChain(memory=self.memory,prompt=prompt,llm=self.llm,verbose=False)
    def _specify_system_message(self):
        """Specify the behaviour of the journalist chatbot.
        The prompt is generated and optimized with GPT-4.
        outputs:
        ----------------
        prompts: instructions for chatbot."""
        prompt=f"""You are a technical journalist interested in {self.topic}, 
            Your task is to distill a recently published scientific paper on this topic through
            an interview with the author, which is played by another chatbot.
            Your objective is to ask comprehensive and technical questions 
            so that anyone who reads the interview can understand the paper's main ideas and contributions, 
            even without reading the paper itself. 
            You're provided with the paper's summary to guide your initial questions.
            You must keep the following guidelines in mind:
            - Focus exclusive on the technical content of the paper.
            - Avoid general questions about {self.topic}, focusing instead on specifics related to the paper.
            - Only ask one question at a time.
            - Feel free to ask about the study's purpose, methods, results, and significance, 
            and clarify any technical terms or complex concepts. 
            - Your goal is to lead the conversation towards a clear and engaging summary.
            - Do not include any prefixed labels like "Interviewer:" or "Question:" in your question.
    
            [Abstract]: {self.abstract}"""
        return prompt
    def step(self,prompt):
        """Journalist chatbot asks question
        
        Args:
        ---------------
        prompt: Previous answer provided by the author bot."""
        response=self.conversation.predict(input=prompt)
        return response

In [5]:
class AuthorBot(Chatbot):
    """Class definition for the author bot,created with Langchain."""
    def __init__(self,engine,vectorstore,debug=False):
        """Select the llm, as well as instantiate the memory for creating language chain in langchain.
        
        Args:
        ----------
        engine: llm model to be used.
        vectorstore: embedding vectors of the pdf."""
        #Instantiate llm
        super().__init__(engine)
        #Intantiate memory
        self.chat_history=[]
        #Instantiate embedding index
        self.vectorstore=vectorstore
        self.debug=debug
    def instruct(self,topic):
        """Determine the context of author chatbot.
        Args:
        -----------
        topic: the topic of the paper."""
        #specify topic
        self.topic=topic
        #define prompt template
        qa_prompt=ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self._specify_system_message()),
            HumanMessagePromptTemplate.from_template("{question}")
        ])
        # Create conversation chain
        self.conversation_qa=ConversationalRetrievalChain.from_llm(llm=self.llm,verbose=self.debug,retriever=self.vectorstore.as_retriever(search_kwargs={'k':5}),return_source_documents=True,combine_docs_chain_kwargs={'prompt':qa_prompt})
    def step(self,prompt):
        """Author chatbot answers question.
        Args:
        ----------
        prompt: question raised by journalist bot.
        
        Outputs:
        ------------
        answer: the author bot's answer
        source_documents: documents that author bot used to answer questions"""
        response=self.conversation_qa({'question':prompt,'chat_history':self.chat_history})
        self.chat_history.append((prompt,response['answer']))
        return response['answer'],response['source_documents']
    def _specify_system_message(self):
        """Specify the behaviour of the author chatbot.
        
        outputs:
        ------------
        prompt: instructions for the chatbot."""
        prompt = f"""You are the author of a recently published scientific paper on {self.topic}.
                You are being interviewed by a technical journalist who is played by another chatbot and
                looking to write an article to summarize your paper.
                Your task is to provide comprehensive, clear, and accurate answers to the journalist's questions.
                Please keep the following guidelines in mind:
                - Try to explain complex concepts and technical terms in an understandable way, without sacrificing accuracy.
                - Your responses should primarily come from the relevant content of this paper, 
                  which will be provided to you in the following, but you can also use your broad knowledge in {self.topic} to 
                  provide context or clarify complex topics. 
                - Remember to differentiate when you are providing information directly from the paper versus 
                  when you're giving additional context or interpretation. Use phrases like 'According to the paper...' for direct information, 
                  and 'Based on general knowledge in the field...' when you're providing additional context.
                - Only answer one question at a time. Ensure that each answer is complete before moving on to the next question.
                - Do not include any prefixed labels like "Author:", "Interviewee:", Respond:", or "Answer:" in your answer.
    """
    
        prompt += """Given the following context, please answer the question.
    
    {context}"""
    
        return prompt

In [9]:
def highlight_PDF(file_path,phrases,output_path):
    """Search and highlight given texts in PDF.
    
    Args:
    ----------------
    file_path: PDF file path
    phrases: a list of texts(in strings)
    output_path: save and output PDF"""
    #Open PDF
    doc=fitz.open(file_path)
    #Search doc
    for page in doc:
        for phrase in phrases:
            text_instances=page.search_for(phrase)
            #Highlight
            for inst in text_instances:
                highlight=page.add_highlight_annot(inst)
    doc.save(output_path,garbage=4)

In [10]:
paper='Vision Transformers'
paper_path="ViT.pdf"
#create embeddings
embedding=Embedder(engine='OpenAI')
embedding.load_process('ViT.pdf')
#set up vectorstore
vectorstore=embedding.create_vectorstore(store_path=paper)
# Fetch paper summary
paper_summary= embedding.create_summary(llm_engine='OpenAI')
# Instantiate journalist and author bot
journalist=JournalistBot('OpenAI')
author= AuthorBot('OpenAI',vectorstore)
#Provide instruction
journalist.instruct(topic='This paper introduces concepts of using transformers for images and computer vision tasks and how tradition transformers adapt to it',abstract=paper_summary)
author.instruct('Paper introducing transformers and the attention mechanism,an architecture which can be widely used for language tasks')
#Start conversation
for i in range(4):
    if i==0:
        question=journalist.step('Start the conversation')
    else:
        question=journalist.step(answer)
    print("👨‍🏫 Journalist: " + question)
    answer, source = author.step(question)
    print("👩‍🎓 Author: " + answer)
    phrases = [src.page_content for src in source]
    highlight_PDF(paper_path, phrases, 'highlighted.pdf')
    page_numbers = [str(src.metadata['page']+1) for src in source]
    unique_page_numbers = list(set(page_numbers))
    print(("For details, please check the highlighted text on page(s): " + ', '.join(unique_page_numbers)))
    

Embeddings found! Loaded the computed ones
👨‍🏫 Journalist: What motivated you to explore the use of transformers, commonly used in natural language processing, for image recognition tasks in your paper?
👩‍🎓 Author: In our paper, we were motivated to explore the use of transformers for image recognition tasks due to the success of transformers in natural language processing (NLP). Transformers have become the standard in NLP tasks because of their computational efficiency and scalability, allowing for training models with unprecedented sizes and achieving state-of-the-art results. We wanted to investigate whether transformers could also excel in computer vision tasks, specifically image recognition, without relying on convolutional neural networks (CNNs), which are traditionally used in this domain. By applying transformers directly to sequences of image patches, we aimed to demonstrate that transformers could perform well in image classification tasks and potentially outperform CNNs wh